![Project Logo](Project_Logo.pdf)

# Build 0.20b (12 Oct 2020)

### Requires Integrator Version No Eariler than *8 Oct 2020*

### This is a Public Release
* Y. Wang: [yourong.f.wang@auckland.ac.nz]

* J. Zagorac
* R. Easther




## PyUltraLight with N-Body Evalutation and Padded Potentials (H-E-B Scheme)


### Note on Exploration
Potential padding and NBody are inplemented separately in this version.

### New and Noteworthy Features
- Rudimentary interaction model between arbitrary point-like matter and ULDM with customizable axion mass.
  - The integrator is currently without an explicit check for **Energy Conservation**
- Field appropriately scaled for dynamics.
- Pre-Computed 2-Body Problem Example Scenarios (Parabola and Circular)
- Improved Movie Plotter Performance (Summary Animation, Time Series Graph, or Individual Panels)
- New 3d Volumetric Visualization (needs plotly)
- Automatic Generation of Output Digest to File




# Disk Space Management:

In [ ]:
import os

save_path = 'FW_NBody_H'  # Set output directory


def get_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

print(save_path,": The current size of the folder is", round(get_size(save_path)/1024**2,3), 'Mib')

if get_size(save_path) == 0:
    cleardir = 'N'
else:
    print(save_path,": Do You Wish to Delete All Files Currently Stored In This Folder? [Y] \n")
    cleardir = str(input())

if cleardir == 'Y':
    import shutil 
    shutil.rmtree(save_path)
    print("Folder Cleaned! \n")

try:
    os.mkdir(save_path)
    print(save_path,": Save Folder Created.")
except FileExistsError:
    if cleardir != 'Y':
        print("")
    else:
        print(save_path,": File Already Exists!")
        
        
def Load_Data():
    
    with open('{}{}'.format(save_path, '/timestamp.txt'), 'r') as timestamp:
        ts = timestamp.read()
        print('PyUL NBody: Loading Folder',ts)

    loc = save_path + '/' + ts


    import time   
    import warnings 
    warnings.filterwarnings("ignore")
    plt.ioff()

    EndNum = 0
    
    global data, TMdata, phidata, graddata, TimeStamp
    
    if save_plane:
        print('PyUL NBody: Loaded Planar Mass Density Data \n')
    if save_testmass:
        print('PyUL NBody: Loaded Test Mass State Data \n')
    if save_phi_plane:
        print('PyUL NBody: Loaded Planar Gravitational Field Data \n')
    if save_gradients:
        print('PyUL NBody: Loaded Test Mass Gradient Data \n')

    TimeStamp = ts
    
    for x in np.arange(0,save_number+1,1):
    #for x in np.arange(0,550,1):    
    
        try:
            if save_plane:

                data.append(np.load('{}{}{}{}'.format(loc, '/plane_#', x, '.npy')))
            if save_testmass:

                TMdata.append(np.load('{}{}{}{}'.format(loc, '/TM_#', x, '.npy')))
            if save_phi_plane:

                phidata.append(np.load('{}{}{}{}'.format(loc, '/Field2D_#', x, '.npy')))
                
            if save_gradients:
                graddata.append(np.load('{}{}{}{}'.format(loc, '/Gradients_#', x, '.npy')))
            
            EndNum += 1
        
        except FileNotFoundError:
        
            TimeStamp = ts
            print("WARNING: Run incomplete or the storage is corrupt!")

            break
        
    print("PyUL NBody: Loaded", EndNum, "Data Entries")
    return EndNum






# Loading Required Packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec
matplotlib.__version__

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation

import PyUltraLight_NBody_HEB as PyUL
print('PyUltralight: Integrator Date', PyUL.D_version())

import pyfftw
import os
import sys
import multiprocessing
import numpy
import numba
import h5py
from matplotlib import cm

import math

from IPython.core.display import clear_output, display
%matplotlib inline

# Set Axion Mass (SI)

In [ ]:
axion_mass = 1e-22 *1.783e-36 #kg

# ULDM Simulation Parameters

In [ ]:
resolN = int(7) # 6 -> 64, 7 -> 128, 8 -> 256, 9 -> 512, 10 -> Explosion

# Set number of threads to target
num_threads = multiprocessing.cpu_count()
print("PyUL NBody: Available CPU threads for this run: ",num_threads)

# Set units for soliton parameters
s_mass_unit = ''     #Accepted units: 'kg', 'solar_masses', 'M_solar_masses', and '' for dimensionless units
s_position_unit = '' #Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units
s_velocity_unit = '' #Accepted units: 'm/s', 'km/s', 'km/h', and '' for dimensionless units

# Rules ditto.
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''

# Set box size and resolution
length = 12 # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
length_units = ''  # Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units.

resol= int(2**resolN)# It is recommended to check the upper bound on soliton mass for a given box size and resolution

print("PyUL NBody: Simulation Grid Size:", resol, 'cubed.')

LengthFS = PyUL.convert(length, length_units, 'l')

GridLenFS = LengthFS/(resol)

GridLenPC = PyUL.convert_back(GridLenFS,'pc','l')

print("PyUL NBody: The grid resolution subdivision is %.6f in specified units. This is %.6f code units." % ( length/resol, GridLenFS))

duration = 1 #1 code unit is ~70 Gyr (independent of axion mass assumption)
duration_units = ''  # Accepted units: 's', 'yr', 'kyr', 'Myr', and '' for dimensionless units
start_time = 0.05 # Should be given in the same units as duration. 

#Data to save
#0
save_rho = False # Saves density data for entire 3D simulation grid
#1
save_psi = False # Saves full complex field data for entire 3D simulation grid
#2
save_plane = True # Saves density data for plane z = 0
#3
save_energies = True # Saves integrated gravitational, kinetic and total energies as lists
#4
save_line = False # Saves density data for line y = 0, z = 0. Useful for examining intereference patterns. 

## FW
#5
save_testmass = True # Saves trajectory and 3-velocity of a test mass thrown into the system.   
#6
save_phi = False
#7
save_phi_plane = True
#8
save_gradients = True

#Formats to save
hdf5 = False
npz = False
npy = True

step_factor = 1 # Change this to a larger number if velocities are sufficiently low that constraint on timestep can be relaxed. 
save_number = 180    # Choose number of 'frames' to save. Note that, depending on resolution, this could require significant disk space.


save_options = [save_rho,save_psi,save_plane,save_energies,save_line,save_testmass,save_phi,save_phi_plane,save_gradients]

PreMult = 0

if save_rho:
    print('PyUL NBody: Saving Mass Density Data (3D)')
    PreMult = PreMult + resol**3
    
if save_psi:
    print('PyUL NBody: Saving Complex Field Data (3D)')
    PreMult = PreMult + resol**3*2
    
if save_phi:
    print('PyUL NBody: Saving Gravitational Field Data (3D)')
    PreMult = PreMult + resol**3
    
if save_plane:
    print('PyUL NBody: Saving Mass Density Data (2D)')
    PreMult = PreMult + resol**2
    
if save_phi_plane:
    print('PyUL NBody: Saving Gravitational Field Data (2D)')
    PreMult = PreMult + resol**2

if save_gradients:
    print('PyUL NBody: Saving NBody Gradient Data')

if save_testmass:
    print('PyUL NBody: Saving NBody Position Data')

SaveSizeEstimate = save_number*(PreMult)*8/(1024**3)

print('PyUL NBody: Estimated Save Size (GiB): %.02f' % SaveSizeEstimate )

# Data_Loader For Later


# ULDM Initial Condition Modifier
If Uniform is set to true, this block overrides all solitons you define

In [ ]:
# Removes Solitons in the IV and replaces them with a uniform wavefunction 
# with given probability amplitude (code unit).

Uniform = False
Density = 1e-6

central_mass = 0 # Not Implemented Anymore. Keep for code compatibility.

 

# Gravitational Field Smoothing
Instead of coding in the idealistic $\frac{m}{r}$ potential, we approximate it with $\frac{am}{ar+e^{-ar}}$. 

The modified potential is a useful approximation of a point mass in a distance, and puts an exponential cap on the infinite potential well, which reduces the impact of numerical artefacts.


The smoothing factor, $a$, is also defined here.


The differences between the two potentials are characterised in the following box.

In [ ]:
a = 15



fig_grav = plt.figure(figsize=(10, 10))

# Diagnostics For Field Smoothing


ax1 = plt.subplot(211)
ax2 = plt.subplot(212)


FS = np.linspace(1,50,50)

rR = GridLenFS*FS

GOrig = -1/rR

GMod = -a*1/(a*rR+np.exp(-a*rR))

GDiff = - GOrig + GMod

# Two little quantifiers
BoundaryEn = next(x for x, val in enumerate(GDiff) if val < 1e-2)
rS = FS[BoundaryEn]

BoundaryEx = next(x for x, val in enumerate(GMod) if val > -a/2)
rQ = FS[BoundaryEx]

ax1.plot(FS,GOrig,'k--',label = 'Point Potential')

ax1.plot([rS,rS],[2,-100],'r.-')
ax1.plot([rQ,rQ],[2,-100],'b.-')


ax1.plot(FS,GMod,'go',label = 'Modified Potential')

ax1.set_ylim([-1.5*a,0])

ax1.legend()

ax1.set_ylabel('$\propto$Energy')
ax1.grid()

ax2.semilogy(FS,(GDiff),'g.')
ax2.set_xlabel('Radial distance from origin (Grids)')
ax2.set_ylabel('Difference')
ax2.semilogy([rS,rS],[1e2,1e-16],'r.-')
ax2.semilogy([rQ,rQ],[1e2,1e-16],'b.-')
ax2.grid()

plt.show()


print('PyUL NBody: Field Smoothing Report:')
print('  The simulation runs on a %.0f^3 grid with total side length %.1f'%(resol,length))
print('  The simulation grid size is %.4f Code Units,\n   which is equivalent to %.1f parsecs' % ( GridLenFS,GridLenPC))

print('\n==========Grid Counts of Important Features=========\n')
print("  Radius outside which the fields are practically indistinguishable (Grids): %.0f" % rS)
print("  Modified Potential HWHM (Grids): %.0f" % rQ)

# Load Example Scenarios:

Just run the following blocks, and "False" scenarios will automatically skip.

Both Parabola and Circular Orbit scenarios come with a great level of customizability, explained below. You can specify the initial positions, masses, and degree of ULDM participation in the dynamics.

If you want to specify your own settings entirely, please set both to False.




## Parabola 

Describes two masses (soliton or BH or hybrid) with equal mass approaching each other on the $x-y$ plane.

You can specify the $(x_0,y_0)$ value, as well as their masses.

The rest of the initial conditions will then be automatically computed.


## Circular Orbit 

Describes two masses (soliton or BH or hybrid) with arbitrary masses orbiting each other on the $x-y$ plane.

You can specify the initial separation, as well as their individual masses.

The rest of the initial conditions will then be automatically computed.

In [ ]:
Parabola = False
Circular = False


## Init
if Parabola:
    

    m = 12.5 #1 code unit is ~2.3e6 M_sol (1e-22/m_a)^1.5
    r = 0.5 #1 code unit is ~38 kpc x (1e-22/m_a)^0.5

    x0 = 1
    y0 = 1
    
    m1 = m
    m2 = m

    # Focal Point Is Origin
    # y^2 = 4cx + 4c^2

    c0 = 1/2*(-x0+np.sqrt(x0**2+y0**2))

    v0 = np.sqrt(m/(2*np.sqrt(x0**2+y0**2))) #Correct

    xDot0 = 1
    yDot0 = 2*c0*xDot0/(y0)

    vNorm = np.linalg.norm([xDot0,yDot0])

    xDot0 = xDot0/vNorm*v0
    yDot0 = yDot0/vNorm*v0

    BH1 = [m,[x0,y0,0],[-xDot0,-yDot0,0]]
    BH2 = [m,[-x0,-y0,0],[xDot0,yDot0,0]]
    
    
    BHT = [0,[0,0,0],[0,0,0]]
    particles = [BH1,BH2]


    #Soliton parameters are mass, position, velocity and phase (radians)
    soliton1 = [m, [x0,y0,0],[-xDot0,-yDot0,0], 0]
    soliton2 = [m, [-x0,-y0,0],[xDot0,yDot0,0], 0]
    solitonE = [1, [0,0,0],[0,0,0], 0]
    solitons = [solitonE]
    # solitons = []

    print("PyUL NBody: Loading 2-Body Parabola Demo")
elif Circular:

    m1 = 50 #1 code unit is ~2.3e6 M_sol (1e-22/m_a)^1.5
    
    m2 = 40
    
    x1 = 0.3
    
    x2 = x1/m2*m1 # Ensures Com Position
    
    xC = x1+x2
    
    yDot1 = np.sqrt(m2*x1/xC**2)
    yDot2 = np.sqrt(m1*x2/xC**2)

           
    BH1 = [m1,[x1,0,0],[0,yDot1,0]]
    BH2 = [m2,[-x2,0,0],[0,-yDot2,0]]
    
    
    particles = [BH1,BH2]


    #Soliton parameters are mass, position, velocity and phase (radians)
    
    soliton1 = [(1-Ratio)*m1,[x1,0,0],[0,yDot1,0], 0]
    soliton2 = [(1-Ratio)*m2,[-x2,0,0],[0,-yDot2,0],0]
    
    solitonC = [10,[0,0,0],[0,0,0],0]

    solitons = [soliton1,soliton2]
    
    
    #solitons = [solitonC]
    print("PyUL NBody: Loading 2-Body Circular Orbit Demo")
    
else:
    
    central_mass = 0
    
    m1 = 0.1
    mS = 2
    
    BH1 = [m1,[2,1,0],[-2,0.5,0]]
    particles = [BH1]
    #Soliton parameters are mass, position, velocity and phase (radians)
    
    soliton1 = [mS, [-0.0,-0.0,0],[0,0,0], 0]
    solitons = [soliton1]
    
    print("PyUL NBody: Loading Custom Settings")


# Execution and Data Loading:

In [ ]:
NumSol = len(solitons)
NumTM = len(particles)
data = []
TMdata = []
phidata = []
graddata = []

try:
    TimeStamp = PyUL.evolve(central_mass, num_threads, length, length_units, 
            resol, duration, duration_units, step_factor, 
            save_number, save_options, save_path, npz, npy, hdf5, 
            s_mass_unit, s_position_unit, s_velocity_unit, solitons,
            start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles,
            Uniform,Density,a)
    
    EndNum = Load_Data()
    
except KeyboardInterrupt:
    print("\n Run Interrupted! The ability to resume simulations is under development.")
    EndNum = Load_Data()

In [ ]:
# Emergency Reloading. For Use only after a Kernel Crash

try:
    print(EndNum)
except NameError:
    EndNum = Load_Data()

# 2D Trajectory Plot

In [ ]:
try:
    TSD = TimeStamp
except NameError:
    TimeStamp = 'Debug'

plt.clf()

fig = plt.figure(figsize=(24, 24))
# debug
import numexpr as ne
import matplotlib as mpl

Zoom2D = 1
Boundary = length/(2*Zoom2D)

ax = fig.add_subplot(111)
mpl.style.use('seaborn')

if Parabola:
    yParaBola = np.linspace(-length/(2),length/(2),resol)
    xParaBola = ne.evaluate("yParaBola**2/(4*c0)-c0")

    plt.plot(xParaBola,yParaBola)

    plt.plot(-xParaBola,yParaBola)
    

if Circular:
    
    tCirc = np.linspace(0,2*np.pi,200)

    plt.plot(x1*np.cos(tCirc),x1*np.sin(tCirc),'-',color = (1,0.5,0,0.1))
    plt.plot(x2*np.cos(tCirc),x2*np.sin(tCirc),'-',color = (1,0,0.5,0.1))



plt.xlim([-Boundary,Boundary])
plt.ylim([-Boundary,Boundary])


ax.set_aspect('equal', adjustable='box')

for i in range(EndNum):
    
    
    TMStateLoc = TMdata[i]
    
    if i == 0:
        for particleID in range(len(particles)):
        
            ColorID = particleID/len(particles) # 0, 0.5
        
            TMx = TMStateLoc[int(6*particleID)]
            TMy = TMStateLoc[int(6*particleID+1)]
            plt.plot([TMx],[TMy],'ko')
        
    
    Alpha = 0.3+7*i/(10*EndNum)
    
    for particleID in range(len(particles)):
        
        ColorID = 0.2 # 0, 0.5
        
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        plt.plot([TMx],[TMy],'.',color=(0.5,1*ColorID,0.6-ColorID,Alpha))
        
ax.grid(True)

textstr = '\n'.join((
    TimeStamp,
    r'Resolution: $%.0f^3$' % (resol, ),
    r'Box Length: %.0f' % (length, ),
    r'Method: %.0f @ %.0f Steps' % (1,1)))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in upper left in axes coords
ax.text(0, 0, textstr, transform=ax.transAxes, fontsize=18,
        verticalalignment='bottom', bbox=props)

plt.show()

# Energy Processing and Plotting

In [ ]:
loc = save_path + '/' + TimeStamp

plt.clf()
# 2Body Energy Savr 

ML = [m1,m2]

# Translating Snapshots back to Time
timerange = np.linspace(0,duration,save_number+1)


KS = np.zeros(int(EndNum))
PS = np.zeros(int(EndNum))

print(len(particles))
for i in range(int(EndNum)):
    
    Data = TMdata[i]
    
    if len(particles)==2:

        r = Data[0:2] - Data[6:8]

        rN = np.linalg.norm(r)
    
        PS[i] = -1*m1*m2*a/(a*rN+np.exp(-1*a*rN))
   
    for particleID in range(len(particles)):
        Vx = Data[int(6*particleID+3)]
        Vy = Data[int(6*particleID+4)]
        Vz = Data[int(6*particleID+5)]
        
        KS[i] = KS[i] + 1/2*ML[particleID]*(Vx**2+Vy**2+Vz**2) 

Tp = np.linspace(0,EndNum,EndNum)

Tp = Tp/EndNum * duration

plt.figure()
plt.plot(Tp,KS,label = '$\sum E_k$')

if len(particles) == 2:
    plt.plot(Tp,PS,label = '$\sum E_p$')
    plt.plot(Tp,PS+KS,'-', label = '$E_k + E_p$')
#plt.plot(Tp,0*Tp,'k--')

plt.title('Energy of Test Masses')
plt.xlabel('Integration Time / $\mathcal{T}$')
plt.ylabel('$\mathcal{M}\mathcal{L}^2\mathcal{T}^{-2}$')

plt.legend()

plt.show()


EnergyName = '{}{}{}{}'.format(save_path,"/EnergyTotal_",TimeStamp,'.jpg')


fig = plt.figure(figsize=(8, 4.5))

egylist = np.load('{}{}'.format(loc, '/egylist.npy')).tolist()
egpcmlist = np.load('{}{}'.format(loc, '/egpcmlist.npy')).tolist()
egpsilist = np.load('{}{}'.format(loc, '/egpsilist.npy')).tolist()
ekandqlist = np.load('{}{}'.format(loc, '/ekandqlist.npy')).tolist()
mtotlist = np.load('{}{}'.format(loc, '/masseslist.npy')).tolist()

plt.plot(Tp,egylist,'g-',label='Total ULDM Energy')
plt.plot(Tp,egpcmlist,'c-.',label='$E_{GP}$ (ULD Potential Due to Test Masses)')
plt.plot(Tp,egpsilist,'m-.',label='$E_{GP}$ (ULD Potential Due to Self-Interaction)')
plt.plot(Tp,ekandqlist,label='ULD $E_{K}+E_{Q}$')


plt.plot(Tp,KS,'y--',label = '$\sum E_k$ for Test Masses')

if len(particles) == 2:
    plt.plot(Tp,PS,'r--',label = '$\sum E_p$ for Test Masses')
    plt.plot(Tp,PS+KS,'b-', label = 'Total Energy of Test Masses')


plt.plot(Tp,PS+KS+egylist,'k-', label = 'Total Energy of Entire System')

plt.legend(ncol=2)

plt.xlabel('Integration Time / $\mathcal{T}$')
plt.ylabel('Net Energy / $\mathcal{M}\mathcal{L}^2\mathcal{T}^{-2}$')

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

plt.savefig(EnergyName, format='jpg', dpi=600)


# plt.xlim([90,100])
plt.show()


EnergyDName = '{}{}{}{}'.format(save_path,"/EnergyDelta_",TimeStamp,'.jpg')

fig = plt.figure(figsize=(16, 9))

Tpt = np.linspace(2,EndNum+1,EndNum-2,endpoint = False) 

Tpt = Tpt/EndNum * duration

egylist = np.array(np.load('{}{}'.format(loc, '/egylist.npy')).tolist())
egpcmlist = np.array(np.load('{}{}'.format(loc, '/egpcmlist.npy')).tolist())
egpsilist = np.array(np.load('{}{}'.format(loc, '/egpsilist.npy')).tolist())
ekandqlist = np.array(np.load('{}{}'.format(loc, '/ekandqlist.npy')).tolist())
mtotlist = np.load('{}{}'.format(loc, '/masseslist.npy')).tolist()


egylistD = egylist - egylist[1]

egpcmlistD = egpcmlist - egpcmlist[1]

egpsilistD = egpsilist - egpsilist[1]

ekandqlistD = ekandqlist - ekandqlist[1]

plt.plot(Tpt,egylistD[1:-1],'g-',label='Total ULDM Energy Within Integration Box')
plt.plot(Tpt,egpcmlistD[1:-1],'c-.',label='$E_{GP}$ (ULD Potential Due to Test Masses)')
plt.plot(Tpt,egpsilistD[1:-1],'m-.',label='$E_{GP}$ (ULD Potential Due to Self-Interaction)')
plt.plot(Tpt,ekandqlistD[1:-1],label='ULD $E_{K}+E_{Q}$')

KSD = KS - KS[1]

PSD = PS - PS[1]

plt.plot(Tpt,KSD[1:-1],'r--',label = '$\sum E_k$ for Test Masses')
    
if len(particles) == 2:
    plt.plot(Tpt,PSD[1:-1]+KSD[1:-1],'b-', label = 'Total Energy of Test Masses')
    
#plt.plot(Tp,0*Tp,'k--')

plt.plot(Tpt,PSD[1:-1]+KSD[1:-1]+egylistD[1:-1],'k-', label = 'Total Energy of Entire System')

plt.legend(frameon=False, ncol=2)

plt.xlabel('Integration Time / $\mathcal{T}$')
plt.ylabel('Energy Change/$\mathcal{M}\mathcal{L}^2\mathcal{T}^{-2}$')

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

plt.savefig(EnergyDName, format='jpg', dpi=600)

#plt.xlim([2,100])


plt.show()

TotalED = PSD+KSD+egylistD


# Summary Animation

In [ ]:
# User Parameters
Loga = False
FPS = 15

BarWidth = 0.6  # the width of the bars

MovieX = 16

MovieY = 16

# File Name Handling


try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(save_path,"/Summary_",VTimeStamp,'.mp4')

print("Saving ",AnimName)


# Defining Grid System and Plotting Variables

NumSol = len(solitons)

figAS = plt.figure(figsize=(MovieX, MovieY))
gs = figAS.add_gridspec(4, 4)

AS_GradGraph = figAS.add_subplot(gs[0, :])
AS_GradGraph.set_title('Acceleration of Particle #1')

AS_FieldPlane = figAS.add_subplot(gs[1:3,0:2])
AS_FieldPlane.set_title('2D Gravitational Field')

AS_RhoPlane = figAS.add_subplot(gs[1:3, 2:4])
AS_RhoPlane.set_title('2D Mass Density')


AS_EDelta = figAS.add_subplot(gs[3, :])
AS_EDelta.set_title('Energy Change Snapshot #1')


AS_FieldPlane.set_aspect('equal')

if Loga:
    
    if Uniform:
        data0 = np.log(np.array(data)/Density)
        print("Initial Field is Uniform. Evaluating Change Ratio.")
    
    else:
        data0 = np.log(data)
        
    planemax = np.max(data0)
    planemin = -50
    
    print("Using Log Plot, the Contour Level Limits Are")
    print(planemax,planemin)
    
else:
    data0 = (data)
    planemax = np.max(data0)
    planemin = np.min(data0)
    
        
levels = np.linspace(planemin, planemax, int(resol/2))

PlotRange = np.linspace(-length/2, length/2,resol,endpoint = False)

BarLabels = ['Total Energy', 'Mass Kinetic Energy', 'ULDM Total Energy']

BarX = np.arange(len(BarLabels))  # the label locations

BarMax = np.max(TotalED)
    
graddataP = (graddata)


DTEMaxChange = 0.
DTEMinChange = 0.

def animateAS(i):

    # Acceleration Graph

    AS_GradGraph.plot(i,graddataP[i][0],'r.',label = '$x$')
    AS_GradGraph.plot(i,graddataP[i][1],'g.',label = '$y$')
    AS_GradGraph.plot(i,graddataP[i][2],'b.',label = '$z$')
   
    
    # Field Graph
    
    sliced = phidata[i]

    AS_FieldPlane.imshow(sliced,origin='lower')

    AS_FieldPlane.set_xticks([])
    AS_FieldPlane.set_yticks([])
    
    # Density Graph
    
    AS_RhoPlane.cla()
    AS_RhoPlane.set_aspect('equal')
    
    AS_RhoPlane.set_xticks([])
    AS_RhoPlane.set_yticks([])
    
    AS_RhoPlane.set_xlim([-length/2,length/2])
    AS_RhoPlane.set_ylim([-length/2,length/2])
    AS_RhoPlane.get_xaxis().set_ticks([])
    AS_RhoPlane.get_yaxis().set_ticks([])
    
    AS_RhoPlane.contour(PlotRange,PlotRange,data0[i], levels=levels, vmin=planemin, vmax=planemax,cmap = 'coolwarm')


    TMStateLoc = TMdata[i]
    for particleID in range(len(particles)):
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+3)]
        Vy = TMStateLoc[int(6*particleID+4)]
        Vz = TMStateLoc[int(6*particleID+5)]
        AS_RhoPlane.plot([TMy],[TMx],'ko')
        AS_RhoPlane.quiver([TMy],[TMx],[Vy],[Vx])
        
        
    # Bar Graph

    AS_EDelta.cla()
    
    DTE = TotalED[i]
    DKE = KSD[i]
    DUE = egylistD[i]
    
    global DTEMaxChange, DTEMinChange
    
    DTEMaxChange = np.max([DTEMaxChange,DTE])
    DTEMinChange = np.min([DTEMinChange,DTE])
    
    EnergyEntry = [DTE, DKE, DUE]
    

    rects1 = AS_EDelta.bar(BarX, EnergyEntry, BarWidth)

    zeroLine = AS_EDelta.plot((-0.5,2.5),(0,0),'k--')
    
    maxLine = AS_EDelta.plot((-0.5,2.5),(DTEMaxChange,DTEMaxChange),'r-.')
    
    minLine = AS_EDelta.plot((-0.5,2.5),(DTEMinChange,DTEMinChange),'b-.')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    AS_EDelta.set_ylabel('Absolute Change Since Onset ($\mathcal{E}$)')
    AS_EDelta.set_ylim(-2*BarMax,2*BarMax)
    AS_EDelta.set_title('Energy Change for Snapshot %.0f'%i)
    AS_EDelta.set_xticks(BarX)
    AS_EDelta.set_xticklabels(BarLabels)


    
    
    
    if i%FPS == 0 and i!= 0:
        print('Animated %.0f seconds out of %.2f seconds of data.' % (i/FPS, EndNum/FPS))
    
    if i == EndNum-1:
        clear_output()
        print('Animation Complete:', AnimName)

        
interval = 0.00001 #in seconds
aniAS = matplotlib.animation.FuncAnimation(figAS,animateAS,EndNum,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

aniAS.save(AnimName, writer=writer)



# Additional Individual Plots

## [Animation] 2D Mass Density

In [ ]:
Loga = False 

try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(save_path,"/Anim2D_",VTimeStamp,'.mp4')

print("Saving ",AnimName)

plot_axis = 'z' #axis that is set to zero
plt.ioff()
fig0, ax0 = plt.subplots(figsize=(16, 9), dpi=72)
ax0.set_aspect('equal')


if Loga:
    
    if Uniform:
        data0 = np.log(np.array(data)/Density)
        print("Initial Field is Uniform. Evaluating Change Ratio.")
    
    else:
        data0 = np.log(data)
    planemax = np.max(data0)
    planemin = np.min(data0)
    
    print("Using Log Plot, the Contour Level Limits Are")
    print(planemax,planemin)
    
else:
    data0 = (data)
    planemax = np.max(data0)
    planemin = np.min(data0)
        
levels = np.linspace(planemin, planemax, int(resol))

PlotRange = np.linspace(-length/2, length/2,resol,endpoint = False)

# print(PlotRange)

FPS = 15


def animate0(i):
    ax0.cla()
    ax0.set_aspect('equal')
    ax0.set_xlim([-length/2,length/2])
    ax0.set_ylim([-length/2,length/2])
    ax0.get_xaxis().set_ticks([])
    ax0.get_yaxis().set_ticks([])
    
    ax0.contour(PlotRange,PlotRange,data0[i], levels=levels, vmin=planemin, vmax=planemax)
    #ax0.imshow(data[i])

    TMStateLoc = TMdata[i]
    for particleID in range(len(particles)):
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+3)]
        Vy = TMStateLoc[int(6*particleID+4)]
        Vz = TMStateLoc[int(6*particleID+5)]
        ax0.plot([TMy],[TMx],'ko')
        ax0.quiver([TMy],[TMx],[Vy],[Vx])
        
    fig0.suptitle('{}{}{}'.format('Mass Density - plane ', plot_axis, '=0'), fontsize = 15)
    ax0.text(0.90, 1.1, '{}{}'.format('Snapshot # ', i), horizontalalignment='center', verticalalignment='center', transform=ax0.transAxes)
    
    if i%FPS == 0 and i!= 0:
        print('Animated %.0f seconds out of %.2f seconds of data.' % (i/FPS, EndNum/FPS))
    
    if i == EndNum-1:
        print('Animation Complete')

interval = 0.15 #in seconds
ani0 = matplotlib.animation.FuncAnimation(fig0,animate0,EndNum,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

ani0.save(AnimName, writer=writer)

## [Animation] Gravitational Field Slice

In [ ]:
NumSol = len(solitons)
try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(save_path,"/Anim3D_",VTimeStamp,'.mp4')

print("Saving ",AnimName)

# Initialization
fig = plt.figure(figsize=(12, 12))
ax = fig.gca(projection='3d')

ax.view_init(90, 0)

PlotRange = np.linspace(-length/2, length/2,resol)

zoom = 0.8

FPS = 15 

x3D, y3D = np.meshgrid(PlotRange,PlotRange)

phidataP = np.array(phidata)*-1

planemax = np.max(phidataP)*1.2
planemin = np.min(phidataP)

print(planemin,planemax)

levels = np.linspace(planemin, planemax, 100)

def animate(i):
    fig.clear()
    global FPS
    
    TMStateLoc = TMdata[i]
    ax = fig.gca(projection='3d')

    ax.view_init(0, 45)

    ax.set_zlim3d(planemin, planemax)
    ax.set_ylim3d(-length/(2*zoom), length/(2*zoom))                    
    ax.set_xlim3d(-length/(2*zoom), length/(2*zoom))

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    if NumSol != 0:
        
        #ax.contour(PlotRange,PlotRange,(phidata[i]),zdir='z', offset=0, cmap=cm.coolwarm, levels = levels)
        

        zs = phidataP[i].flatten()
        ax.plot_wireframe(x3D,y3D,phidataP[i],rcount = resol, ccount = resol)
 
    
    for particleID in range(len(particles)):
        TMx = TMStateLoc[int(6*particleID)]
        TMy = TMStateLoc[int(6*particleID+1)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+3)]
        Vy = TMStateLoc[int(6*particleID+4)]
        Vz = TMStateLoc[int(6*particleID+5)]
        ax.plot([TMy],[TMx],[TMz],'ko')
        ax.quiver([TMy],[TMx],[TMz],[Vy],[Vx],[Vz],length=length/(resol*zoom), normalize=False)
    #ax.plot([TMS[i,0]],[TMS[i,1]],[TMS[i,2]],'ro')
    
        ax.text(TMy, TMx, TMz, '{}{}{}'.format(particleID,'Particle Snapshot # ', i))

    
    if i%FPS == 0 and i!= 0:
        print('Animated %.0f seconds out of %.2f seconds of data.' % (i/FPS, EndNum/FPS))
        
    if i == EndNum-1:
        print('Animation Complete')
        
    plt.draw() 

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

interval = 0.05 #in seconds


ani = matplotlib.animation.FuncAnimation(fig,animate,EndNum,interval=interval*1e+3,blit=False)

ani.save(AnimName, writer=writer)
 

# Acceleration of Particles

In [ ]:
pID = 0 # 0,1,2,...

plt.clf()
plt.figure(figsize=(16, 9))

graddataP = (graddata)
for i in range(EndNum):
    pID = pID*3
    plt.plot(i,graddataP[i][0],'r.',label = '$x$')
    plt.plot(i,graddataP[i][1],'g.',label = '$y$')
    plt.plot(i,graddataP[i][2],'b.',label = '$z$')

plt.xlabel('Snapshot Number')
plt.ylabel('Particle Acceleration')
plt.legend()
plt.show()

## Rate Estimate of Energy Change

In [ ]:
timerange = np.linspace(0,duration,save_number+1)

TotalED = PSD+KSD+egylistD

EDFit = np.polyfit(timerange,TotalED,1)

print(EDFit)

MaxED = max(TotalED.min(), TotalED.max(), key=abs)

FinalED = TotalED[-1]

print(FinalED, MaxED)

plt.figure(figsize = (16,9))

plt.plot(timerange,TotalED,'k-')

plt.plot(timerange,EDFit[0]*timerange+EDFit[1],'r--')

plt.show()

## [Bonus Interactive]: 3D Field Visualization

Loads all data into RAM. Very slow.

In [ ]:
if save_rho:
    
    with open('{}{}'.format(save_path, '/timestamp.txt'), 'r') as timestamp:
        ts = timestamp.read()
        print('PyUL NBody: Loading Folder',ts)

    loc = save_path + '/' + ts

    import time   
    import warnings 
    warnings.filterwarnings("ignore")
    plt.ioff()

    EndNum = 0
    Rhodata = []
    TMdata = []

    for x in np.arange(0,save_number+1,1):
    #for x in np.arange(0,550,1):    

        try:

            Rhodata.append(np.load('{}{}{}{}'.format(loc, '/rho_#', x, '.npy')))
            TMdata.append(np.load('{}{}{}{}'.format(loc, '/TM_#', x, '.npy')))
            EndNum += 1
        except FileNotFoundError:

            TimeStamp = ts
            print("PyUL NBody: Run incomplete or the storage is corrupt!")

            break

    print("PyUL NBody: Loaded", EndNum, "Data Entries. Which one to plot?")
    
    PlotNum = int(input())

    import plotly.graph_objects as go
    
    import plotly.io as pio
    
    import numpy as np
    
    if Uniform:
        Rhodata = np.log(np.array(Rhodata))
        
    

    HL = length/2
    XP, YP, ZP = np.mgrid[-HL:HL:resol/4*1j, -HL:HL:resol/4*1j, -HL:HL:resol/4*1j]

    print(len(XP))
    values = Rhodata[PlotNum]
    
    values = values.flatten()
    
    shrink = int(resol/4)**3
    
    Values = np.zeros(shrink)
    
    for ii in range(shrink):
        Values[ii] = values[int(ii*4)]

    fig3D = go.Figure(data=go.Volume(
        x=XP.flatten(),
        y=YP.flatten(),
        z=ZP.flatten(),
        value=Values,
        isomin=np.min(Rhodata),
        isomax=np.max(Rhodata),
        opacity=0.02, # needs to be small to see through all surfaces
        surface_count=int(resol/4), # needs to be a large number for good volume rendering
        ))

else:
    raise RuntimeError ("You did not save field data!")

In [ ]:
import plotly.io as pio
pio.write_html(fig3D, 'Trial.html')

## [Bonus Interactive]: 3D Density Visualization

In [ ]:
print(phidataP[1])

In [ ]:
import plotly.graph_objects as go
 
import plotly.io as pio
import numpy as np

import pandas as pd

phidataPL = pd.DataFrame(data=phidataP[287], index=None, columns=None)

fig = go.Figure(data=[go.Surface(z=phidataPL.values)])

#fig.update_layout(title='$z=0$', autosize=False,width=500, height=500,margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
pio.write_html(fig, 'Soliton512.html')

In [ ]:
print(phidataPL)

In [ ]:
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
print(z_data)